# AILib Tutorial 4: Prompt Builder

The PromptBuilder is a powerful tool for constructing complex, multi-turn conversations programmatically. In this tutorial, you'll learn:

- Building conversations step by step
- Adding different message types
- Managing conversation context
- Advanced prompt construction techniques
- Integration with LLM clients
- Real-world conversation patterns

## Setup

Let's import what we need:

In [ ]:
from ailib import OpenAIClient
from ailib.prompts import PromptBuilder, PromptTemplate
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Create a client
client = OpenAIClient()
print("Ready to build prompts!")

## Basic Prompt Building

The PromptBuilder helps you construct conversations programmatically:

In [ ]:
# Create a simple conversation
builder = PromptBuilder()

# Add messages
builder.add_system("You are a helpful coding assistant.")
builder.add_user("What is a Python decorator?")

# Get the built messages
messages = builder.build()
print("Built messages:")
for msg in messages:
    print(f"- {msg['role']}: {msg['content'][:50]}...")

# Use with client
response = client.chat(messages)
print(f"\nResponse: {response}")

## Building Multi-Turn Conversations

PromptBuilder excels at managing conversation history:

In [ ]:
# Create a multi-turn conversation
builder = PromptBuilder()

# Set the system context
builder.add_system("You are a Python tutor who teaches through examples.")

# First turn
builder.add_user("How do I use list comprehensions?")
builder.add_assistant("List comprehensions provide a concise way to create lists. Here's the basic syntax:\n\n[expression for item in iterable if condition]")

# Second turn
builder.add_user("Can you show me some examples?")

# Get response for the conversation so far
messages = builder.build()
response = client.chat(messages)
print("Assistant response:")
print(response)

# Add the response and continue
builder.add_assistant(response)
builder.add_user("How about nested list comprehensions?")

# Get the next response
messages = builder.build()
final_response = client.chat(messages)
print("\n" + "="*50 + "\n")
print("Final response:")
print(final_response)

## Method Chaining

PromptBuilder supports fluent API for cleaner code:

In [ ]:
# Build a conversation using method chaining
messages = (PromptBuilder()
    .add_system("You are a creative writing assistant.")
    .add_user("Help me write a haiku about programming")
    .add_assistant("I'd be happy to help you write a haiku about programming! Here's one:\n\nCode flows like water\nBugs surface, then disappear\nShip it anyway")
    .add_user("That's funny! Can you write a more serious one?")
    .build()
)

response = client.chat(messages)
print(response)

## Working with Templates

Combine PromptBuilder with PromptTemplate for dynamic conversations:

In [ ]:
# Create templates for different parts
system_template = PromptTemplate(
    "You are a {profession} specializing in {specialty}. You communicate in a {style} manner."
)

user_template = PromptTemplate(
    "I need help with {task}. My current level is {level}."
)

# Build a personalized conversation
def create_expert_conversation(profession, specialty, style, task, level):
    return (PromptBuilder()
        .add_system(system_template.format(
            profession=profession,
            specialty=specialty,
            style=style
        ))
        .add_user(user_template.format(
            task=task,
            level=level
        ))
        .build()
    )

# Create different expert conversations
scenarios = [
    ("data scientist", "machine learning", "technical but accessible", "understanding neural networks", "beginner"),
    ("chef", "Italian cuisine", "warm and encouraging", "making homemade pasta", "intermediate"),
    ("fitness coach", "strength training", "motivational", "building a workout routine", "beginner")
]

for profession, specialty, style, task, level in scenarios:
    messages = create_expert_conversation(profession, specialty, style, task, level)
    response = client.chat(messages)
    print(f"\n{profession.title()} Response:")
    print(response[:200] + "...\n")
    print("=" * 50)

## Managing Conversation State

PromptBuilder helps manage conversation state effectively:

In [ ]:
class ConversationManager:
    """Manages ongoing conversations with state."""
    
    def __init__(self, system_prompt):
        self.builder = PromptBuilder()
        self.builder.add_system(system_prompt)
        self.turn_count = 0
    
    def add_user_message(self, message):
        """Add a user message to the conversation."""
        self.builder.add_user(message)
        self.turn_count += 1
    
    def add_assistant_message(self, message):
        """Add an assistant message to the conversation."""
        self.builder.add_assistant(message)
    
    def get_response(self, client):
        """Get a response from the LLM."""
        messages = self.builder.build()
        response = client.chat(messages)
        self.add_assistant_message(response)
        return response
    
    def get_summary(self):
        """Get a summary of the conversation."""
        return {
            "turns": self.turn_count,
            "messages": len(self.builder.build()),
            "last_message": self.builder.build()[-1]['content'][:100] + "..."
        }

# Use the conversation manager
manager = ConversationManager(
    "You are a helpful travel advisor who provides personalized recommendations."
)

# Have a conversation
manager.add_user_message("I'm planning a trip to Japan. What should I know?")
response1 = manager.get_response(client)
print("Response 1:")
print(response1[:200] + "...\n")

manager.add_user_message("What about food recommendations in Tokyo?")
response2 = manager.get_response(client)
print("Response 2:")
print(response2[:200] + "...\n")

# Get conversation summary
print("Conversation Summary:")
print(manager.get_summary())

## Advanced Prompt Construction

Build complex prompts with dynamic content:

In [ ]:
# Dynamic prompt construction based on context
def build_code_review_prompt(code, language, focus_areas, severity_level="standard"):
    """Build a comprehensive code review prompt."""
    builder = PromptBuilder()
    
    # System prompt based on severity
    if severity_level == "strict":
        builder.add_system(
            f"You are a senior {language} developer performing a strict code review. "
            "Be thorough and point out even minor issues."
        )
    else:
        builder.add_system(
            f"You are an experienced {language} developer providing constructive code review."
        )
    
    # Add context
    context = f"Please review this {language} code focusing on: {', '.join(focus_areas)}"
    builder.add_user(context)
    
    # Add the code
    code_message = f"```{language}\n{code}\n```"
    builder.add_user(code_message)
    
    # Add specific questions based on focus areas
    if "performance" in focus_areas:
        builder.add_user("Pay special attention to any performance bottlenecks.")
    
    if "security" in focus_areas:
        builder.add_user("Highlight any potential security vulnerabilities.")
    
    return builder.build()

# Example code to review
python_code = """
def process_user_input(user_data):
    # Process user data
    query = f"SELECT * FROM users WHERE name = '{user_data['name']}'"
    results = database.execute(query)
    
    processed_data = []
    for row in results:
        processed_data.append(row)
    
    return processed_data
"""

# Build review prompt
messages = build_code_review_prompt(
    code=python_code,
    language="Python",
    focus_areas=["security", "best practices", "performance"],
    severity_level="strict"
)

# Get review
review = client.chat(messages)
print("Code Review:")
print(review)

## Conversation Patterns

Common patterns for effective conversations:

In [ ]:
# Pattern 1: Clarification Loop
def clarification_conversation(initial_request, client):
    """Build a conversation that clarifies user requirements."""
    builder = PromptBuilder()
    builder.add_system(
        "You are a requirements analyst. Ask clarifying questions to fully understand the user's needs."
    )
    builder.add_user(initial_request)
    
    # Get initial response with questions
    messages = builder.build()
    response = client.chat(messages)
    
    return response, builder

# Example
request = "I need a function to process data"
questions, builder = clarification_conversation(request, client)
print("Clarifying questions:")
print(questions)

# Simulate user providing more details
builder.add_assistant(questions)
builder.add_user(
    "The data is CSV files with sales information. I need to calculate totals by region and product."
)

# Get refined response
final_response = client.chat(builder.build())
print("\n" + "="*50 + "\n")
print("Refined response:")
print(final_response[:300] + "...")

In [ ]:
# Pattern 2: Progressive Refinement
def progressive_refinement(initial_output, refinements, client):
    """Progressively refine output based on feedback."""
    builder = PromptBuilder()
    builder.add_system("You are a creative assistant who refines work based on feedback.")
    
    # Initial request
    builder.add_user("Write a short product description for a smart home thermostat.")
    builder.add_assistant(initial_output)
    
    # Apply refinements
    for refinement in refinements:
        builder.add_user(f"Please refine this: {refinement}")
        messages = builder.build()
        response = client.chat(messages)
        builder.add_assistant(response)
        print(f"After '{refinement}':")
        print(response)
        print("\n" + "-"*30 + "\n")
    
    return builder.build()[-1]['content']

# Initial output
initial = "SmartTemp: The intelligent thermostat that learns your preferences."

# Refinements
refinements = [
    "Make it more technical and detailed",
    "Add energy saving benefits",
    "Make it more compelling for homeowners"
]

final = progressive_refinement(initial, refinements, client)
print("Final version:")
print(final)

## Contextual Prompts

Build prompts that adapt to context:

In [ ]:
class ContextualPromptBuilder:
    """Builds prompts with rich context."""
    
    def __init__(self):
        self.builder = PromptBuilder()
        self.context = {}
    
    def set_context(self, **kwargs):
        """Set context variables."""
        self.context.update(kwargs)
    
    def build_with_context(self, template_string):
        """Build a prompt using context variables."""
        # Create template from string
        template = PromptTemplate(template_string)
        
        # Format with context
        formatted = template.format(**self.context)
        
        # Add to builder
        self.builder.add_user(formatted)
        
        return self.builder.build()

# Use contextual builder
ctx_builder = ContextualPromptBuilder()

# Set up context
ctx_builder.set_context(
    user_name="Alice",
    project="E-commerce Website",
    tech_stack="React, Node.js, MongoDB",
    deadline="2 weeks",
    priority="performance optimization"
)

# Add system message
ctx_builder.builder.add_system(
    "You are a senior software architect providing technical guidance."
)

# Build contextual prompt
messages = ctx_builder.build_with_context(
    "Hi, I'm {user_name} working on {project} using {tech_stack}. "
    "I have {deadline} to complete this project. "
    "My main concern is {priority}. What should I focus on?"
)

response = client.chat(messages)
print("Contextual Response:")
print(response)

## Real-World Example: Interactive Tutorial System

Let's build a complete interactive tutorial system:

In [ ]:
class InteractiveTutor:
    """An interactive tutorial system using PromptBuilder."""
    
    def __init__(self, subject, client):
        self.subject = subject
        self.client = client
        self.builder = PromptBuilder()
        self.current_topic = None
        self.difficulty = "beginner"
        
        # Initialize system prompt
        self.builder.add_system(
            f"You are an expert {subject} tutor. You teach through clear explanations, "
            "examples, and practice problems. Adapt to the student's level and provide "
            "encouragement. Always check understanding before moving on."
        )
    
    def start_lesson(self, topic, difficulty="beginner"):
        """Start a new lesson on a topic."""
        self.current_topic = topic
        self.difficulty = difficulty
        
        prompt = f"Let's learn about {topic}. The student is at {difficulty} level."
        self.builder.add_user(prompt)
        
        response = self._get_response()
        return response
    
    def ask_question(self, question):
        """Student asks a question."""
        self.builder.add_user(question)
        return self._get_response()
    
    def request_example(self):
        """Request an example."""
        self.builder.add_user(
            f"Can you show me a practical example of {self.current_topic}?"
        )
        return self._get_response()
    
    def practice_problem(self):
        """Request a practice problem."""
        self.builder.add_user(
            f"Give me a {self.difficulty} level practice problem for {self.current_topic}."
        )
        return self._get_response()
    
    def check_solution(self, solution):
        """Check a solution to a practice problem."""
        self.builder.add_user(f"Here's my solution: {solution}")
        return self._get_response()
    
    def _get_response(self):
        """Get response from the LLM."""
        messages = self.builder.build()
        response = self.client.chat(messages)
        self.builder.add_assistant(response)
        return response

# Create a Python tutor
tutor = InteractiveTutor("Python programming", client)

# Start a lesson
print("Starting lesson...")
print("=" * 50)
intro = tutor.start_lesson("list comprehensions", "intermediate")
print(intro)

# Ask for an example
print("\n" + "=" * 50)
print("Requesting example...")
example = tutor.request_example()
print(example)

# Get a practice problem
print("\n" + "=" * 50)
print("Getting practice problem...")
problem = tutor.practice_problem()
print(problem)

## Best Practices

Tips for effective use of PromptBuilder:

In [ ]:
# 1. Keep conversations focused
def focused_conversation():
    return (PromptBuilder()
        .add_system("You are a Git expert. Only answer questions about Git.")
        .add_user("How do I undo the last commit?")
        .build()
    )

# 2. Provide clear context
def contextual_conversation():
    return (PromptBuilder()
        .add_system("You are helping debug a web application.")
        .add_user("Context: React app, using hooks, getting 'Invalid Hook Call' error")
        .add_user("The error appears when I click the submit button")
        .build()
    )

# 3. Structure complex prompts
def structured_conversation():
    builder = PromptBuilder()
    
    # Clear sections
    builder.add_system("You are a technical writer creating documentation.")
    
    # Background
    builder.add_user("Project: REST API for task management")
    
    # Requirements
    builder.add_user("Requirements: Clear, concise, with examples")
    
    # Specific request
    builder.add_user("Document the POST /tasks endpoint")
    
    return builder.build()

# 4. Manage conversation length
class ConversationLimiter:
    def __init__(self, max_turns=10):
        self.builder = PromptBuilder()
        self.max_turns = max_turns
        self.turn_count = 0
    
    def add_turn(self, user_msg, assistant_msg):
        if self.turn_count >= self.max_turns:
            # Summarize and reset
            self._summarize_and_reset()
        
        self.builder.add_user(user_msg)
        self.builder.add_assistant(assistant_msg)
        self.turn_count += 1
    
    def _summarize_and_reset(self):
        # In practice, you'd summarize the conversation
        print("Conversation limit reached, summarizing...")
        self.builder = PromptBuilder()
        self.builder.add_system("Previous conversation summarized. Continue helping the user.")
        self.turn_count = 0

print("Best practices examples created!")

## Summary

In this tutorial, you learned:

- ✅ How to build conversations step by step with PromptBuilder
- ✅ How to add different message types (system, user, assistant)
- ✅ How to use method chaining for cleaner code
- ✅ How to combine PromptBuilder with PromptTemplate
- ✅ How to manage conversation state and context
- ✅ Common conversation patterns and best practices
- ✅ How to build interactive systems with PromptBuilder

PromptBuilder is essential for:
- Building complex, multi-turn conversations
- Managing conversation history
- Creating reusable conversation patterns
- Building interactive AI applications

## Next Steps

Continue learning with:

- **Tutorial 5: Session Management** - Persist conversations across requests
- **Tutorial 6: Chains** - Connect multiple operations
- **Tutorial 7: Tools and Decorators** - Extend agent capabilities

Happy building! 🏗️